<a href="https://colab.research.google.com/github/skyfuryonline/semeval/blob/master/debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [13]:
from datasets import Dataset
from transformers import AutoTokenizer

# 示例数据
data = {
        "source_locale": ["en","en"],
        "target_locale": ["fr","fr"],
        "source": ["How many Star Wars movies has Chewbacca appeared in?","'Good!' said Norbert, 'there's one of my father's spies gone; that leaves only the little cripple Napier.'"],
        "target": ["Combien de films Star Wars Chewbacca est-il apparu?",'– Bon! dit Norbert, voilà un des espions de mon père parti ; il ne reste plus que le petit boiteux Napier.'],
}

# 创建Dataset对象
dataset = Dataset.from_dict(data)
print("-------------------------------")
print(dataset[0:2])
print("-------------------------------")

column_name = dataset.column_names


# 加载 T5 模型的分词器
model_id = "google-t5/t5-small"  # 使用 T5-small 模型
tokenizer = AutoTokenizer.from_pretrained(model_id)

-------------------------------
{'source_locale': ['en', 'en'], 'target_locale': ['fr', 'fr'], 'source': ['How many Star Wars movies has Chewbacca appeared in?', "'Good!' said Norbert, 'there's one of my father's spies gone; that leaves only the little cripple Napier.'"], 'target': ['Combien de films Star Wars Chewbacca est-il apparu?', '– Bon! dit Norbert, voilà un des espions de mon père parti ; il ne reste plus que le petit boiteux Napier.']}
-------------------------------


In [8]:
# 任务说明和输入格式
prefix = "translate English to French: "

# 预处理函数
def preprocess_function(examples):
    # 为每个句子添加任务描述前缀
    inputs = [prefix + example for example in examples['source']]
    targets = [example for example in examples['target']]
    # 使用 T5 分词器对输入和目标进行编码
    model_inputs = tokenizer(inputs, text_target=targets, max_length=64, truncation=True, padding="max_length")
    return model_inputs

# 预处理数据
tokenized_data = dataset.map(preprocess_function, batched=True,remove_columns=column_name)

# 查看预处理后的数据
print(tokenized_data[0])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

{'input_ids': [13959, 1566, 12, 2379, 10, 571, 186, 2042, 10265, 4876, 65, 2556, 210, 9305, 658, 4283, 16, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [2570, 4232, 29, 20, 4852, 2042, 10265, 2556, 210, 9305, 658, 259, 18, 173, 3, 10198, 76, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
